In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import copy
# import albumentations as A
import random
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import imageio
from imgaug.augmentables.batches import Batch
from PIL import Image

In [2]:
aug = iaa.SomeOf(3,[
    iaa.MultiplyBrightness((0.6, 1.5)),
    iaa.GaussianBlur(sigma=(0, 3.0)),
    iaa.LogContrast(gain=(0.8, 1.0)),
    iaa.Affine(rotate=(-20, 20), mode = "edge"),
    iaa.CoarseDropout((0.1, 0.3), size_percent=(0.01, 0.03), per_channel=0.2),   
], random_order=True)

In [3]:
data_aug = "../dataset/data_aug"
data_aug_json = "../dataset/data_aug_json"
if not os.path.isdir(data_aug):
    os.makedirs(data_aug)
if not os.path.isdir(data_aug_json):
    os.makedirs(data_aug_json)


In [4]:
img_aug_bbox ={}
img_path = "../dataset/train_set"
total_imgs = os.listdir("../dataset/train_set")
total_imgs = sorted(total_imgs, key= lambda s : int(os.path.splitext(os.path.basename(s))[0]), reverse = False)

with open("../dataset/train_set_json/bbox.json","r") as bbox_file:
    bbox = json.load(bbox_file)

aug_index = 0

for i in range(0, len(total_imgs)):
    img = imageio.imread(os.path.join(img_path, total_imgs[i]))
    x,y,w,h = tuple(bbox[str(i)])
    bb = BoundingBox(x,y,x+w,y+h)
    bb_ = BoundingBoxesOnImage([bb], shape=img.shape)
    for j in range(0 ,4):
        image_aug, bbs_aug = aug(image=img, bounding_boxes=bb_)
        int_xyxy = [int(x) for x in bbs_aug.to_xyxy_array()[0]]
        if int_xyxy[0] < 0:
            int_xyxy[0] = 0
        if int_xyxy[1] < 0:
            int_xyxy[1] = 0
        if int_xyxy[2] > 539 :
            int_xyxy[2] = 539
        if int_xyxy[3] > 959:
            int_xyxy[3] = 959
        aug_x, aug_y, auug_w, aug_h = int_xyxy[0], int_xyxy[1], int_xyxy[2] - int_xyxy[0], int_xyxy[3] - int_xyxy[1]
        
        img_aug_bbox[aug_index] = [aug_x, aug_y, auug_w, aug_h]
        imageio.imwrite(os.path.join( data_aug, "{index}.png".format(index = aug_index)),image_aug)
        
        aug_index += 1
        
with open(os.path.join( data_aug_json, "bbox.json"),"w") as img_aug_bbox_file:
    json.dump( img_aug_bbox, img_aug_bbox_file, indent=4)

C:\Users\user\anaconda3\envs\eardetection\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  if sys.path[0] == "":
